In [3]:
import json
import requests
import numpy as np
with open('data.json', 'r', encoding='utf-8') as json_file:
    inp = json.load(json_file)

In [ ]:
url = "http://reg.gst.gov.in/registration/"
response = requests.get(url)
cookie=response.headers['Set-Cookie'].split(';')[0]

In [6]:
def create_captcha1:
    url = f"https://reg.gst.gov.in/services/captcha?rnd={np.random.random()}"
    headers = {
        "Host": "reg.gst.gov.in",
        "Cookie": cookie,
        "Accept": "image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8",
        "Referer": "https://reg.gst.gov.in/registration/",
        }
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open("captcha.png", "wb") as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
        print("Captcha image saved as captcha.png")
    else:
        print("Failed to fetch captcha image")
    captchacookie = response.headers['Set-Cookie'].split('; ')[0]
    return captchacookie

In [405]:
captcha="552202"

In [406]:

url = "https://reg.gst.gov.in/registration/api/submit/sendotp"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": f"{cookie}; {captchacookie}",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in"
}
data={'applnType': 'APLRG',
 'pt': 'P',
 'stCd': inp['opdtls'][0]['rsad']['stcd'],
 'dtCd': inp['opdtls'][0]['rsad']['dst'],
 'lgBzName': inp['opdtls'][0]['fn']+" "+inp['opdtls'][0]['mn']+" "+inp['opdtls'][0]['ln'],
 'id': inp['opdtls'][0]['pan']['num'],
 'email': inp['opdtls'][0]['em'],
 'mbno': inp['opdtls'][0]['mbno'],
 'captcha': captcha}
response = requests.post(url, headers=headers, data=json.dumps(data))

print("Response Status Code:", response.status_code)
print("Response Content:")
if not 'invalid' in json.loads(response.text)['message']:
    print(response.text)


Response Status Code: 200
Response Content:
{"stscd":"SUCCESS","message":"OTP sent successfully","errors":""}


In [407]:
del data['captcha']
data["smsOtp"]="089379"
data["emailOtp"]="145742"

In [410]:
import requests
import json

url = "https://reg.gst.gov.in/registration/api/submit/submitotp"

headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": f"{cookie}; {captchacookie}",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in"
}
response = requests.post(url, headers=headers, data=json.dumps(data))
trn = json.loads(response.text)['trn']


Response Status Code: 200
Response Content:
{"trn":"292300175912TRN","stscd":"SUCCESS","message":"New Registration","errors":"REG1011"}


In [412]:
url = f"https://reg.gst.gov.in/services/captcha?rnd={np.random.random()}"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": cookie,
    "Accept": "image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8",
    "Referer": "https://reg.gst.gov.in/registration/",
    }
response = requests.get(url, headers=headers, stream=True)
if response.status_code == 200:
    with open("captcha1.png", "wb") as f:
        for chunk in response.iter_content(chunk_size=128):
            f.write(chunk)
    print("Captcha image saved as captcha1.png")
else:
    print("Failed to fetch captcha image")
captchacookie = response.headers['Set-Cookie'].split('; ')[0]

Captcha image saved as captcha1.png


In [415]:
captcha = "534970"

In [416]:
url = "https://reg.gst.gov.in/registration/api/submit/sendtrnotp"

headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": f"{cookie}; {captchacookie}",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in"
}
data={"trn":f"{trn}","captcha":captcha}
response = requests.post(url, headers=headers, data=json.dumps(data))


Response Status Code: 200
Response Content:
{"stscd":"SUCCESS","message":"OTP sent successfully","errors":""}


In [418]:
smsotp = "480565"

In [419]:
url = "https://reg.gst.gov.in/registration/api/submit/submittrnotp"

headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": f"{cookie}",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in"
}
data={"trn":f"{trn}","smsOtp": smsotp}
response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.text)
auth_token=response.headers['set-cookie'].split(';')[0].split('=')[1]

Response Status Code: 200
Response Content:
{"stscd":"SUCCESS","message":"New Registration","errors":"REG1011"}


In [421]:
new_cookie=f"{cookie};AuthToken={auth_token};"

In [424]:
url = "https://reg.gst.gov.in/registration/api/get/ExstPanDtls"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": cookie,
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://reg.gst.gov.in",
 }
data1 = {
    "pan": inp['opdtls'][0]['pan']['num'],
    "stateCode": inp['opdtls'][0]['rsad']['stcd']
}
response = requests.post(url, headers=headers, data=json.dumps(data1))

print(response.text)


Response Status Code: 200
Response Content:
{"pan":"GGUPP1603Q","panDate":"Pan date not available","regDtls":[{"arn":"AA290723061198W","arnstatus":"Pending for Clarification","flag":false,"modlInd":"M2","taxPyrTyp":"Normal","stateCd":"29","arnDt":"06/07/2023","assignTo":"STATE","statusRsn":"Clarification sought by Tax officer and time to provide clarification is not yet over. Task is pending with the taxpayer."}]}


In [425]:
pandate=json.loads(response.text)['panDate']

In [427]:
url = "https://reg.gst.gov.in/registration/auth/api/myapp"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": new_cookie,
    }
response = requests.get(url, headers=headers)
print(response.text)
draft_id=eval(response.text)[0]['draft_id']
inp['aplCd']=draft_id


Response Status Code: 200
Response Content:
[{"creation_dt":"28/08/2023","form_no":"GST REG-01","expiry_dt":"12/09/2023","form_desc":"Application for New Registration","draft_id":48738513,"version_id":1,"state_cd":"29","status":"Draft","appcd":"APLRG"}]


In [430]:
#to upload photograph of promoter
url = "https://reg.gst.gov.in/document"
headers = {
    "Cookie": new_cookie,
    "X-Requested-With": "XMLHttpRequest",
    "Origin": "https://reg.gst.gov.in",
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/promoters"
}

files = {
    "upfile": ("vethika_photo.jpeg", open(r"C:\Users\Admin\Downloads\gst_vdp\vethika_photo.jpeg", "rb"), "image/jpeg"),
}

data1 = {
    "ty": "PHOT",
    "applnid": draft_id
}

response = requests.post(url, headers=headers, files=files, data=data1)
print("Response Content:", response.text)


Response Status Code: 200
Response Content: {"id":106533340,"ty":"PHOT","ct":"image/jpeg","hash":"c0f38c244f7dd232032d44d629923914ecc952712a5e467d6747d975aa45e559"}


In [433]:
#auth signatory letter of auth file upload
    
url = "https://reg.gst.gov.in/document"
headers = {
    "Cookie": new_cookie,
    "Origin": "https://reg.gst.gov.in",
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/authsignatory"
    }

files = {
    "upfile": ("GST-Declaration-Authorised-Signatory_HIGH (1).pdf", open(r"gst_vdp/GST-Declaration-Authorised-Signatory_HIGH (1).pdf", "rb"), "application/pdf"),
}

data1 = {
    "ty": "LOAU",
    #"ty":"CRBC" if Copy of resolution passed by BoD / Managing Committee
    "applnid": draft_id
}

response = requests.post(url, headers=headers, files=files, data=data1)
print("Response Content:", response.text)
auth_sign_att1=json.loads(response.text)


Response Status Code: 200
Response Content: {"id":106533349,"ty":"LOAU","ct":"application/pdf","hash":"5ea7edf32c6254e2cb80bf1175ad9b6829c9009b221a3dcd6f1f8d9a0fddfb3e"}


In [435]:
#auth signatory photo file upload
#limit file size to 100kb
url = "https://reg.gst.gov.in/document"
headers = {
    "Cookie": new_cookie,
    "Origin": "https://reg.gst.gov.in",
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/authsignatory"
    }
files = {
    "upfile": (r"kashvi_pic.jpeg", open(r"gst_vdp/kashvi_pic.jpeg", "rb"), "image/jpeg"),
}
data1 = {
    "ty": "PHOT",
    "applnid": draft_id
}
response = requests.post(url, headers=headers, files=files, data=data1)
print("Response Content:", response.text)
auth_sign_att2=json.loads(response.text)


Response Status Code: 200
Response Content: {"id":106533350,"ty":"PHOT","ct":"image/jpeg","hash":"35c089d09ddaf3ec24969282d57ca812f5532a3dd331229db7f2b2741b95782b"}


In [436]:
inp['asgdtls'][0]['dcupdtls']=[auth_sign_att1,auth_sign_att2]


In [438]:
url = "https://reg.gst.gov.in/documenthb/upload"
headers = {
    "Cookie": new_cookie,
    "Origin": "https://reg.gst.gov.in",
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/business/place"
}

files = {
    "upfile": ("electricty_bill.jpeg", open(r"gst_vdp/electricty_bill.jpeg", "rb"), "image/jpeg"),
}
data1 = {
    "ty": "ELCB",
    "applnid": draft_id
}

response = requests.post(url, headers=headers, files=files, data=data1)

print("Response Status Code:", response.status_code)
print("Response Content:", response.text)
id=json.loads(response.text)['id']
url = f"https://reg.gst.gov.in/documenthb/docstatus/{id}"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": new_cookie,
    "Accept": "application/json, text/plain, */*",
    "At": auth_token,
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/business/place",
   }
response = requests.get(url, headers=headers)
print("Response Content:", response.text)
ppz_doc_att1=json.loads(response.text)


Response Status Code: 200
Response Content: {"status":"WAIT","id":"20230829144066"}
Response Status Code: 200
Response Content: {"id":"20230829144066","ty":"ELCB","ct":"image/jpeg","docName":"electricty_bill.jpeg","hash":"f980c1f360d88746b46768cec7c2d8074a284ab4d557843ad747f712e41892e8"}


In [440]:
url = "https://reg.gst.gov.in/documenthb/upload"
headers = {
    "Cookie": new_cookie,
    "Origin": "https://reg.gst.gov.in",
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/business/place"
}
files = {
    "upfile": ("NOC_high__1__compressed.pdf", open(r"gst_vdp/NOC_high__1__compressed.pdf", "rb"), "application/pdf"),
}
data1 = {
    "ty": "CNLR",
    "applnid": draft_id
}
response = requests.post(url, headers=headers, files=files, data=data1)
print("Response Content:", response.text)
id=json.loads(response.text)['id']
url = f"https://reg.gst.gov.in/documenthb/docstatus/{id}"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": new_cookie,
    "Accept": "application/json, text/plain, */*",
    "At": auth_token,
    "Referer": "https://reg.gst.gov.in/registration/auth/newappl/business/place",
   }
response = requests.get(url, headers=headers)

print("Response Status Code:", response.status_code)
print("Response Content:", response.text)
ppz_doc_att2=json.loads(response.text)


Response Status Code: 200
Response Content: {"status":"WAIT","id":"20230829144067"}
Response Status Code: 200
Response Content: {"id":"20230829144067","ty":"CNLR","ct":"application/pdf","docName":"NOC_high__1__compressed.pdf","hash":"15557f9b735ae2516b23a3acb3024dac1fca86d3bad4b1b0db12a1f479cf90fe"}


In [441]:
inp['ppbzdtls']['dcupdtls']=[ppz_doc_att1,ppz_doc_att2]
inp['secToken']=trn
inp['stsidtls']={}
inp['adhrflag']="Y"
inp['asgdtls'][0]['aadhaarflag']="Y"
inp['opdtls'][0]['aadhaarflag']="Y"


'GGUPP1603Q'

In [448]:
inp['decdtls']={'asdes': inp['opdtls'][0]['dg'], 'pl': inp['opdtls'][0]['rsad']['loc'], 'dt': '28/08/2023', 'asnm': inp['opdtls'][0]['pan']['num']}


In [449]:
url = "https://reg.gst.gov.in/registration/auth/api/save/appdraftnewreg"
headers = {
    "Host": "reg.gst.gov.in",
    "Cookie": new_cookie,
    "Content-Type": "application/json;charset=UTF-8",
    "At": auth_token,
    "Origin": "https://reg.gst.gov.in",
   }
response = requests.post(url, headers=headers, data=json.dumps(inp))
print(response.text)



Response Status Code: 200
Response Content:
{"aplId":"48738513","stscd":"SUCCESS","errors":""}


In [452]:
ck=response.headers['Set-Cookie'].split(',')[-1].split(';')[0]

In [453]:
new_cookie1 = f'AuthToken={auth_token};{ck}'

In [20]:
inp['opdtls'][0]['mbno']

'7353239757'

In [465]:
import requests

url = 'https://reg.gst.gov.in/registration/auth/api/evc/otp'

headers = {
    'Host': 'reg.gst.gov.in',
    'Cookie': new_cookie1,
    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json;charset=UTF-8',
    'At': auth_token,
    'Origin': 'https://reg.gst.gov.in',
    'Referer': 'https://reg.gst.gov.in/registration/auth/newappl/verify',
    }

data = {
    "formType":"APLRG","tranId":trn,"userName":inp['opdtls'][0]['fn']+" "+inp['opdtls'][0]['mn']+" "+inp['opdtls'][0]['ln'],"email":inp['opdtls'][0]['em'],"mobNo":inp['opdtls'][0]['mbno']
}

response = requests.post(url, json=data, headers=headers)
print("Response Content:", response.content)
tranid=json.loads(response.text)['message']


Response Status Code: 200
Response Content: b'{"stscd":"1","message":"5877271ef1fd426b8443be9c4c336e45","errors":""}'


In [ ]:
otp=""

In [469]:
import requests

url = 'https://reg.gst.gov.in/registration/auth/api/evc/submit'
headers = {
    'Host': 'reg.gst.gov.in',
    'Cookie': new_cookie,
    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json;charset=UTF-8',
    'At': auth_token,
    'Origin': 'https://reg.gst.gov.in',
    'Referer': 'https://reg.gst.gov.in/registration/auth/newappl/verify',
    }
data = {
    "otp": otp,
    "dataToSign": json.dumps(inp),
    "formType": "APLRG",
    "tranId": tranid,
    "userId": trn
}

response = requests.post(url, json=data, headers=headers)
print("Response Content:", response.content)


Response Status Code: 200
Response Content: b'{"stscd":"1","message":"SUCCESS","errors":""}'
